In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from finvizfinance.screener.overview import Overview
from scipy.stats import linregress
import time

In [2]:
# ==========================================
# 1.(Finviz Screening)
# ==========================================
def get_filtered_picks(limit_per_country=5000): # Raised limit to capture more
    print("--- STEP 1: Fetching Universe from Finviz ---")
    
    try:
        foverview = Overview()
        
        # Criteria: USA, Strong Buy, High Liquidity, Not Penny Stocks
        filters_dict = {
            
            'Country': 'USA',
            'Average Volume': 'Over 2M', 
            'Market Cap.': '+Small (over $300mln)',
            # 1. Institutional Floor (Removes Penny Stocks)
            'Price': 'Over $5',
    
            # 2. Legitimacy Check (Ensures Options Market exists)
            'Option/Short': 'Optionable',
    
            # 3. Solvency Check (Basic Liquidity) consider removing since it filters out telecom and utilities companies
            'Current Ratio': 'Over 1',
    
            # 4. Sector Exclusion (If you want to remove volatility) thinking about using later
            # 'Industry': 'Stocks only (ex-Funds)',

            #5 profitability filter
            'Operating Margin': 'Positive (>0%)',
            
            #6 Analyst Rating Filter consider changing later
            'Analyst Recom.': 'Strong Buy (1)',
        }
        foverview.set_filter(filters_dict=filters_dict)
        df_results = foverview.screener_view()
        
        if df_results.empty:
            return pd.DataFrame()
            
        print(f"   Success! Found {len(df_results)} initial candidates.")
        
        # Clean up Column Names immediately
        if 'Analyst Recom' in df_results.columns:
            df_results.rename(columns={'Analyst Recom': 'Recom'}, inplace=True)
            
        # Ensure numeric Price for later
        df_results['Price'] = pd.to_numeric(df_results['Price'], errors='coerce')
        
        return df_results

    except Exception as e:
        print(f"   Error in Finviz Step: {e}")
        return pd.DataFrame()

In [3]:
# ==========================================
# 1B. SPECIALIZED UNIVERSE: FINANCIALS (USA + CANADA)
# ==========================================
def get_financial_universe():
    print("--- STEP 1 (Financials): Fetching Banking & Insurance Universe (USA + Canada) ---")
    
    # Same logic: High volume for USA, Lower volume for dual-listed Canadian banks
    targets = [
        {'country': 'USA', 'vol': 'Over 2M'},      
        {'country': 'Canada', 'vol': 'Over 200K'}  
    ]
    
    combined_frames = []
    
    for target in targets:
        c_name = target['country']
        print(f"   Querying {c_name} Financial Sector...")
        
        try:
            foverview = Overview()
            
            # --- DISTINCT FILTERS FOR BANKS ---
            filters_dict = {
                'Country': c_name,
                'Sector': 'Financial',             # <--- CRITICAL: ONLY FINANCIALS
                'Average Volume': target['vol'], 
                'Market Cap.': '+Small (over $300mln)',
                'Price': 'Over $5',
                'Option/Short': 'Optionable',      # Quality filter
            }
            
            foverview.set_filter(filters_dict=filters_dict)
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                print(f"      -> Found {len(df_results)} financial stocks in {c_name}.")
                combined_frames.append(df_results)
            else:
                print(f"      -> No financial results for {c_name}.")
                
        except Exception as e:
            print(f"      Error fetching {c_name}: {e}")

    # Combine data
    if not combined_frames:
        return pd.DataFrame()
        
    fins_universe = pd.concat(combined_frames, ignore_index=True)
    
    # Clean up Column Names
    if 'Analyst Recom' in fins_universe.columns:
        fins_universe.rename(columns={'Analyst Recom': 'Recom'}, inplace=True)
        
    # Ensure numeric Price
    fins_universe['Price'] = pd.to_numeric(fins_universe['Price'], errors='coerce')
    
    # Remove duplicates
    fins_universe.drop_duplicates(subset=['Ticker'], inplace=True)
    
    print(f"   TOTAL FINANCIAL UNIVERSE: {len(fins_universe)} stocks.")
    return fins_universe

In [4]:
# ==========================================
# 2. THE CREDIT MODEL (Z-Score & Margins)
# ==========================================
def calculate_z_score(info, financials, balance_sheet):
    """Calculates Altman Z-Score (Bankruptcy Risk)"""
    try:
        # Extract Key Metrics
        total_assets = balance_sheet.loc['Total Assets'].iloc[0]
        total_liab = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
        current_assets = balance_sheet.loc['Current Assets'].iloc[0]
        current_liab = balance_sheet.loc['Current Liabilities'].iloc[0]
        
        # Z-Score Components
        working_capital = current_assets - current_liab
        retained_earnings = balance_sheet.loc['Retained Earnings'].iloc[0] if 'Retained Earnings' in balance_sheet.index else 0
        
        # Handle EBIT naming differences
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan
            
        market_cap = info.get('marketCap', 0)
        sales = financials.loc['Total Revenue'].iloc[0]

        # Ratios
        A = working_capital / total_assets
        B = retained_earnings / total_assets
        C = ebit / total_assets
        D = market_cap / total_liab
        E = sales / total_assets

        # Formula: 1.2A + 1.4B + 3.3C + 0.6D + 1.0E Altman Z-Score coefficients
        z_score = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z_score, 2)
    except:
        return np.nan
#The Margin Trend column is a custom metric we built to measure operational momentum. 
# It tells you whether a company is becoming more or less efficient at making money over time.
def get_margin_trend(financials):
    """Calculates 3-Year Gross Margin Slope"""
    try:
        years = financials.columns[:3]
        margins = []
        for date in years:
            rev = financials.loc['Total Revenue'][date]
            profit = financials.loc['Gross Profit'][date]
            if rev == 0 or np.isnan(rev): margins.append(0)
            else: margins.append(profit/rev)
            
        # Slope Calculation
        margins = margins[::-1] # Chronological order
        slope, _, _, _, _ = linregress(range(len(margins)), margins)
        
        if slope > 0.005: return "Improving"
        elif slope < -0.005: return "Deteriorating"
        else: return "Stable"
    except:
        return "N/A"


In [5]:
# ==========================================
# 3. EXECUTION ENGINE (SPLIT BY TIER)
# ==========================================
import time

def run_screener_split():
    # A. Get the List from Step 1
    candidates = get_filtered_picks()
    
    if candidates.empty:
        print("No stocks found to analyze.")
        return None, None, None, None, None

    print(f"\n--- STEP 2: Running Credit Risk Model on {len(candidates)} Stocks ---")
    print("Separating stocks into distinct tiers...")
    
    # Initialize separate lists for each tier
    fortress_data = []
    moonshot_data = []
    distress_data = []
    financial_data = [] # Banks/Insurance (skipped by Z-score)
    middle_data = []    # Everything else
    
    # Loop through ALL candidates
    for index, row in candidates.iterrows():
        ticker = row['Ticker']
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # 1. SEPARATE FINANCIALS
            # Z-Score doesn't work for banks, so we stash them here to analyze later
            sector = info.get('sector', 'Unknown')
            if 'Financial' in sector:
                financial_data.append({
                    'Ticker': ticker,
                    'Sector': sector,
                    'Price': row['Price'],
                    'Recom': row.get('Recom', 'N/A')
                })
                continue
            
            # 2. FETCH DATA
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty:
                continue
            
            # 3. RUN METRICS (Using your helper functions)
            z_score = calculate_z_score(info, fin, bs)
            trend = get_margin_trend(fin)
            
            # Data packet to save
            stock_data = {
                'Ticker': ticker,
                'Company': row.get('Company', 'N/A'),
                'Z-Score': z_score,
                'Margin_Trend': trend, 
                'Price': row['Price'],
                'Recom': row.get('Recom', 'N/A'),
                'Sector': sector
            }
            
            # 4. SORT INTO LISTS (The Logic)
            if (z_score > 2.99) and (trend in ["Improving", "Stable"]):
                fortress_data.append(stock_data)
                
            elif (z_score < 1.8) and (trend == "Deteriorating"):
                distress_data.append(stock_data)
                
            elif (z_score < 1.8) and (trend == "Improving"):
                moonshot_data.append(stock_data)
            
            else:
                middle_data.append(stock_data)
            
            # Progress indicator
            if index % 10 == 0:
                print(f"   Processed {index} / {len(candidates)}...", end='\r')

        except Exception:
            continue

    # B. Convert Lists to DataFrames
    fortress_df = pd.DataFrame(fortress_data)
    moonshot_df = pd.DataFrame(moonshot_data)
    distress_df = pd.DataFrame(distress_data)
    financial_df = pd.DataFrame(financial_data)
    middle_df = pd.DataFrame(middle_data)
    
    print(f"\n\n--- ANALYSIS COMPLETE ---")
    print(f"Fortress (Safe):     {len(fortress_df)} stocks")
    print(f"Moonshot (Risky):    {len(moonshot_df)} stocks")
    print(f"Distress (Avoid):    {len(distress_df)} stocks")
    print(f"Financials (Skip):   {len(financial_df)} stocks")
    print(f"Middle of Road:      {len(middle_df)} stocks")
    
    return fortress_df, moonshot_df, distress_df, financial_df, middle_df

# --- RUN IT & UNPACK VARIABLES ---
# This saves the results into 5 separate dataframes immediately
fortress_df, moonshot_df, distress_df, fins_df, mid_df = run_screener_split()
#fins_df are banking and other financial sector stocks cannot use Z score due to customer deposits being liabilities

--- STEP 1: Fetching Universe from Finviz ---
   Success! Found 70 initial candidates.###--------] 3/4 

--- STEP 2: Running Credit Risk Model on 70 Stocks ---
Separating stocks into distinct tiers...
   Processed 60 / 70...

--- ANALYSIS COMPLETE ---
Fortress (Safe):     26 stocks
Moonshot (Risky):    6 stocks
Distress (Avoid):    3 stocks
Financials (Skip):   10 stocks
Middle of Road:      25 stocks


In [6]:
if fortress_df is not None and not fortress_df.empty:
    print("--- FORTRESS STOCKS (High Quality) ---")
    
    display(fortress_df.sort_values(by='Price', ascending=True))


--- FORTRESS STOCKS (High Quality) ---


,Ticker,Company,Z-Score,Margin_Trend,Price,Recom,Sector
22,TTI,"Tetra Technologies, Inc",3.28,Improving,9.30,N/A,Industrials
18,RELY,Remitly Global Inc,6.12,Improving,14.02,N/A,Technology
0,ADMA,Adma Biologics Inc,21.12,Improving,19.13,N/A,Healthcare
14,MIR,Mirion Technologies Inc,3.43,Improving,23.66,N/A,Industrials
6,CPNG,Coupang Inc,4.17,Improving,24.53,N/A,Consumer Cyclical
20,SLB,SLB Ltd,3.06,Improving,37.92,N/A,Energy
3,APG,APi Group Corporation,3.09,Improving,39.22,N/A,Industrials
19,SEI,Solaris Energy Infrastructure Inc,4.85,Improving,44.74,N/A,Energy
4,BROS,Dutch Bros Inc,4.41,Improving,64.48,N/A,Consumer Cyclical
11,KO,Coca-Cola Co,4.44,Improving,70.30,N/A,Consumer Defensive


In [7]:
#high risk picks
if moonshot_df is not None and not moonshot_df.empty:
    print("--- MOONSHOT STOCKS (Speculative) ---")
    display(moonshot_df.sort_values(by='Price', ascending=True))

--- MOONSHOT STOCKS (Speculative) ---


,Ticker,Company,Z-Score,Margin_Trend,Price,Recom,Sector
2,MNKD,Mannkind Corp,-7.27,Improving,5.76,N/A,Healthcare
5,XERS,Xeris Biopharma Holdings Inc,-0.28,Improving,7.40,N/A,Healthcare
0,ET,Energy Transfer LP,1.34,Improving,16.25,N/A,Energy
1,MGNI,Magnite Inc,0.73,Improving,16.43,N/A,Communication Services
4,PRMB,Primo Brands Corp,0.93,Improving,16.64,N/A,Consumer Defensive
3,PGY,Pagaya Technologies Ltd,1.21,Improving,21.30,N/A,Technology


In [8]:
#very high risk / avoid picks
if distress_df is not None and not distress_df.empty:
    print("--- DISTRESS (Avoid / Short Candidates) ---")
    display(distress_df.sort_values(by='Z-Score', ascending=True))

--- DISTRESS (Avoid / Short Candidates) ---


,Ticker,Company,Z-Score,Margin_Trend,Price,Recom,Sector
0,BULL,Webull Corp,-0.51,Deteriorating,7.99,N/A,Technology
2,PAYO,Payoneer Global Inc,0.23,Deteriorating,5.66,N/A,Technology
1,CWH,Camping World Holdings Inc,1.64,Deteriorating,9.87,N/A,Consumer Cyclical


In [9]:
""""# ==========================================
# 4. SPECIALIZED FINANCIALS ENGINE (UPDATED)
# ==========================================
def run_financials_analysis(df):
    
    Takes the raw list of Financial stocks (Banks/Insurance),
    fetches valuation metrics, and filters for Value & Income.
    
    if df is None or df.empty:
        print("No financial stocks to analyze.")
        return

    print(f"--- ANALYZING {len(df)} FINANCIAL STOCKS (USA + TSX) ---")
    print("Fetching P/E, P/B, Dividend, and Analyst Ratings...\n")
    
    bank_data = []
    
    for index, row in df.iterrows():
        ticker = row['Ticker']
        try:
            # Fetch Data
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # 1. VALUATION METRICS
            pe = info.get('trailingPE', np.nan)
            pb = info.get('priceToBook', np.nan)
            
            # 2. EFFICIENCY & INCOME
            roe = info.get('returnOnEquity', np.nan)
            div_yield = info.get('dividendYield', 0)
            if div_yield is None: div_yield = 0
            
            # 3. GROWTH (Revenue Growth)
            rev_growth = info.get('revenueGrowth', np.nan)

            # Save it (Added 'Recom' here)
            bank_data.append({
                'Ticker': ticker,
                'Company': row.get('Company', ticker),
                'Price': row['Price'],
                'P/E': pe,
                'P/B': pb,
                'ROE': roe,
                'Yield%': round(div_yield * 100, 2),
                'Recom': row.get('Recom', 3.0),  # Defaults to 3.0 (Hold) if missing
                'Rev_Growth': rev_growth,
                'Sector': row.get('Sector', 'Financial')
            })
            
        except Exception:
            continue
            
    # Create the Enriched DataFrame
    rich_fins_df = pd.DataFrame(bank_data)
    
    if rich_fins_df.empty:
        print("Could not fetch details for financial stocks.")
        return

    # ==========================================
    # FILTER 1: "UNDERVALUED BANKING" (The Value Play)
    # Criteria: Cheap (P/E < 15), Trading near assets (P/B < 1.2), Profitable (ROE > 8%)
    # ==========================================
    value_picks = rich_fins_df[
        (rich_fins_df['P/E'] < 15) & 
        (rich_fins_df['P/B'] < 1.2) & 
        (rich_fins_df['ROE'] > 0.08)
    ].copy()
    
    print(f"--- FILTER 1: UNDERVALUED BANKS (P/E < 15 & P/B < 1.2) ---")
    if not value_picks.empty:
        display(value_picks.sort_values(by='P/B', ascending=True))
    else:
        print("No stocks met the strict value criteria.")

    # ==========================================
    # FILTER 2: "INCOME COMPOUNDERS" (The Dividend Play)
    # Criteria: High Yield (>2.5%), Growing (>0%), Sustainable P/E (<20)
    # PLUS: Analyst Buy Rating (Score <= 2.5)
    # ==========================================
    income_picks = rich_fins_df[
        (rich_fins_df['Yield%'] >= 2.0) & 
        (rich_fins_df['P/E'] < 20) &
        (rich_fins_df['ROE'] > 0.05) &
        (rich_fins_df['Recom'] <= 2.5) # <--- NEW FILTER (1=Strong Buy, 2=Buy)
    ].copy()

    print(f"\n--- FILTER 2: INCOME GENERATORS (Yield > 3.5% & Buy Rating) ---")
    if not income_picks.empty:
        display(income_picks.sort_values(by='Yield%', ascending=False))
    else:
        print("No stocks met the dividend criteria.")

    return rich_fins_df"""

'"# ==========================================\n# 4. SPECIALIZED FINANCIALS ENGINE (UPDATED)\n# ==========================================\ndef run_financials_analysis(df):\n    \n    Takes the raw list of Financial stocks (Banks/Insurance),\n    fetches valuation metrics, and filters for Value & Income.\n    \n    if df is None or df.empty:\n        print("No financial stocks to analyze.")\n        return\n\n    print(f"--- ANALYZING {len(df)} FINANCIAL STOCKS (USA + TSX) ---")\n    print("Fetching P/E, P/B, Dividend, and Analyst Ratings...\n")\n    \n    bank_data = []\n    \n    for index, row in df.iterrows():\n        ticker = row[\'Ticker\']\n        try:\n            # Fetch Data\n            stock = yf.Ticker(ticker)\n            info = stock.info\n            \n            # 1. VALUATION METRICS\n            pe = info.get(\'trailingPE\', np.nan)\n            pb = info.get(\'priceToBook\', np.nan)\n            \n            # 2. EFFICIENCY & INCOME\n            roe = info.get(\

In [10]:
# ==========================================
# 4. SPECIALIZED FINANCIALS ENGINE (FIXED)
# ==========================================
def run_financials_analysis(df):
    """
    Takes the raw list of Financial stocks (Banks/Insurance),
    fetches valuation metrics, and filters for Value & Income.
    """
    if df is None or df.empty:
        print("No financial stocks to analyze.")
        return

    print(f"--- ANALYZING {len(df)} FINANCIAL STOCKS (USA + TSX) ---")
    print("Fetching P/E, P/B, Dividend, and Analyst Ratings...\n")
    
    bank_data = []
    
    for index, row in df.iterrows():
        ticker = row['Ticker']
        try:
            # Fetch Data
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # 1. VALUATION METRICS
            pe = info.get('trailingPE', np.nan)
            pb = info.get('priceToBook', np.nan)
            
            # 2. EFFICIENCY & INCOME
            roe = info.get('returnOnEquity', np.nan)
            div_yield = info.get('dividendYield', 0)
            if div_yield is None: div_yield = 0
            
            # 3. ANALYST RATING (The Fix)
            # We fetch directly from YF to ensure we get a number (1=Strong Buy, 5=Sell)
            recom = info.get('recommendationMean', None)
            
            # Fallback: If no analyst covers it, assume 3.0 (Hold) so it doesn't crash
            if recom is None: 
                recom = 3.0

            # 4. GROWTH
            rev_growth = info.get('revenueGrowth', np.nan)

            # Save it
            bank_data.append({
                'Ticker': ticker,
                'Company': row.get('Company', ticker),
                'Price': row['Price'],
                'P/E': pe,
                'P/B': pb,
                'ROE': roe,
                'Yield%': round(div_yield * 100, 2),
                'Recom': float(recom),  # Ensure it is a float for filtering
                'Rev_Growth': rev_growth,
                'Sector': row.get('Sector', 'Financial')
            })
            
        except Exception:
            continue
            
    # Create the Enriched DataFrame
    rich_fins_df = pd.DataFrame(bank_data)
    
    if rich_fins_df.empty:
        print("Could not fetch details for financial stocks.")
        return

    # ==========================================
    # FILTER 1: "UNDERVALUED BANKING" (The Value Play)
    # Criteria: Cheap (P/E < 15), Trading near assets (P/B < 1.2), Profitable (ROE > 8%)
    # ==========================================
    value_picks = rich_fins_df[
        (rich_fins_df['P/E'] < 15) & 
        (rich_fins_df['P/B'] < 1.2) & 
        (rich_fins_df['ROE'] > 0.08)
    ].copy()
    
    print(f"--- FILTER 1: UNDERVALUED BANKS (P/E < 15 & P/B < 1.2) ---")
    if not value_picks.empty:
        display(value_picks.sort_values(by='P/B', ascending=True))
    else:
        print("No stocks met the strict value criteria.")

    # ==========================================
    # FILTER 2: "INCOME COMPOUNDERS" (The Dividend Play)
    # Criteria: High Yield (>2.5%), Growing (>0%), Sustainable P/E (<20)
    # PLUS: Analyst Buy Rating (Score <= 2.5)
    # ==========================================
    income_picks = rich_fins_df[
        (rich_fins_df['Yield%'] >= 2.5) & 
        (rich_fins_df['P/E'] < 20) &
        (rich_fins_df['ROE'] > 0.05) &
        (rich_fins_df['Recom'] <= 2.0)  #consider removing or changing later
    ].copy()

    print(f"\n--- FILTER 2: INCOME GENERATORS (Yield > 2.5% & Buy Rating) ---")
    if not income_picks.empty:
        display(income_picks.sort_values(by='Yield%', ascending=False))
    else:
        print("No stocks met the dividend criteria.")

    return rich_fins_df

In [11]:
# ==========================================
# MAIN EXECUTION: FINANCIALS TRACK
# ==========================================

# 1. Fetch the data (Uses the function from Part 1B)
# Note: This checks USA (>2M Volume) and Canada (>200K Volume)
fins_df = get_financial_universe()

# 2. Analyze the data (Uses the function from Part 4)
# This will display the tables for "Undervalued Banks" and "Income Generators"
if not fins_df.empty:
    final_banks_df = run_financials_analysis(fins_df)
else:
    print("No financial stocks found to analyze.")

--- STEP 1 (Financials): Fetching Banking & Insurance Universe (USA + Canada) ---
   Querying USA Financial Sector...
      -> Found 85 financial stocks in USA.###------] 4/5 
   Querying Canada Financial Sector...
      -> Found 8 financial stocks in Canada.
   TOTAL FINANCIAL UNIVERSE: 93 stocks.
--- ANALYZING 93 FINANCIAL STOCKS (USA + TSX) ---
Fetching P/E, P/B, Dividend, and Analyst Ratings...

--- FILTER 1: UNDERVALUED BANKS (P/E < 15 & P/B < 1.2) ---


,Ticker,Company,Price,P/E,P/B,ROE,Yield%,Recom,Rev_Growth,Sector
56,MARA,MARA Holdings Inc,9.43,3.657549,0.689901,0.23066,0.0,1.92308,0.917,Financial
13,BMNR,BitMine Immersion Technologies Inc,28.39,2.108663,0.762366,0.08016,4.0,1.00000,0.942,Financial
61,OBDC,Blue Owl Capital Corp,12.68,9.014113,0.853414,0.09772,888.0,1.58333,0.116,Financial
17,BXSL,Blackstone Secured Lending Fund,27.31,10.290565,1.004568,0.10021,1131.0,3.00000,0.045,Financial
4,ARCC,Ares Capital Corp,20.25,10.178391,1.012598,0.10061,950.0,3.00000,0.009,Financial
62,ONB,Old National Bancorp,22.83,13.432352,1.104527,0.08265,244.0,2.00000,0.478,Financial



--- FILTER 2: INCOME GENERATORS (Yield > 2.5% & Buy Rating) ---


,Ticker,Company,Price,P/E,P/B,ROE,Yield%,Recom,Rev_Growth,Sector
61,OBDC,Blue Owl Capital Corp,12.68,9.014113,0.853414,0.09772,888.0,1.58333,0.116,Financial
77,USB,U.S. Bancorp,54.56,12.483982,1.501941,0.11720,371.0,2.00000,0.072,Financial
79,VLY,Valley National Bancorp,11.92,13.551818,0.910977,0.07067,368.0,1.64286,0.263,Financial
44,HBAN,"Huntington Bancshares, Inc",17.62,12.328671,1.318328,0.10434,349.0,1.66667,0.148,Financial
36,FITB,Fifth Third Bancorp,47.94,14.314926,1.639207,0.11511,331.0,1.81818,0.067,Financial
65,PNC,PNC Financial Services Group Inc,212.65,13.724984,1.567015,0.11486,318.0,1.91304,0.108,Financial
21,CFG,Citizens Financial Group Inc,59.24,16.592438,1.077549,0.06714,308.0,1.52381,0.136,Financial
57,MET,Metlife Inc,80.02,15.074388,1.827261,0.12798,283.0,1.76471,-0.059,Financial
90,RY,Royal Bank Of Canada,172.13,16.763388,2.588880,0.15295,273.0,1.75000,0.138,Financial
38,FNB,F.N.B. Corp,17.42,12.446428,0.941027,0.07870,273.0,1.37500,0.105,Financial


In [12]:
import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['GRND', 'ARCC'] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df['Close'].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100 

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    '52W_MA': round(ma_52w, 2),          # <--- Added Here
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            master_df = df_finviz
            
        # Added '52W_MA' to this list so it displays in the final table
        cols = ['Ticker', 'Price', 'Change_%', '52W_MA', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 2 stocks ---
1. Fetching Analyst Ratings from Finviz...
2. Fetching Price & Volatility from yfinance...


C:\Users\James\AppData\Local\Temp\ipykernel_17288\72418538.py:40: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)



--- Final Watchlist ---


,Ticker,Price,Change_%,52W_MA,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
1,GRND,13.39,-0.74,17.66,-46.72,1.40,21.75,0.27,3.18
0,ARCC,20.25,0.27,20.48,-9.53,1.27,22.64,0.80,1.38
